<a href="https://colab.research.google.com/github/jeffeuxMartin/ColabIPython/blob/main/%E3%80%8CNew_New_maml_omniglot_ipynb2%E3%80%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Go_to_main](#mainprog)

In [ ]:
import torch
try:
    # Get GPU name, check if it's K80
    GPU_name = torch.cuda.get_device_name()
    if GPU_name[-3:] == "K80":
        print("Get K80! :'( RESTART!")
        exit()  # Restart the session
    else:
        print("Your GPU is {}!".format(GPU_name))
        print("Great! Keep going~")
except RuntimeError as e:
    if e.args == ("No CUDA GPUs are available",):
        print("You are training with CPU! "
              "Please restart!")
        exit()  # Restart the session
    else:
        print("What's wrong here?")
        print("Error message: \n", e)

Your GPU is Tesla T4!
Great! Keep going~


In [ ]:
!nvidia-smi


Thu Apr 29 08:39:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if 1:
    workspace_dir = '.'

    # gdown 是一個可以從 google drive 下載資料的工具
    # gdown is a package that downloads files from       \
    #     google drive
    !gdown --id 1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U \
        --output "{workspace_dir}/Omniglot.tar.gz"

    # 使用 tar 解壓縮
    # Use `tar' command to decompress
    !tar -zxf "{workspace_dir}/Omniglot.tar.gz"          \
        -C "{workspace_dir}/"

Downloading...
From: https://drive.google.com/uc?id=1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U
To: /content/Omniglot.tar.gz
26.4MB [00:00, 123MB/s] 


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""      Created on Sat Apr 17 04:51:56 2021
         @author: Jeff Chen                       """;

In [ ]:
""" >>> Construct the Model """;

In [ ]:
# Import modules we need
import glob
from collections import OrderedDict

import numpy as np
from tqdm import tqdm

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from PIL import Image
from IPython.display import display

In [ ]:
def ConvBlock(in_ch, out_ch):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, 3, padding=1),
        nn.BatchNorm2d(out_ch),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))

def ConvBlockFunction(x, w, b, w_bn, b_bn):
    x = F.conv2d(x, w, b, padding=1)
    x = F.batch_norm(x,
                     running_mean=None,
                     running_var=None,
                     weight=w_bn, bias=b_bn,
                     training=True)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)
    return x

class Classifier(nn.Module):
    def __init__(self, in_ch, k_way):
        super(Classifier, self).__init__()
        self.conv1 = ConvBlock(in_ch, 64)
        self.conv2 = ConvBlock(64, 64)
        self.conv3 = ConvBlock(64, 64)
        self.conv4 = ConvBlock(64, 64)
        self.logits = nn.Linear(64, k_way)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.shape[0], -1)
        x = self.logits(x)
        return x
    def functional_forward(self, x, params):
        '''
        Arguments:
        x: input images [batch, 1, 28, 28]
        params: 模型的參數，也就是 convolution 的 weight
                跟 bias，以及 batch normalization 的
                weight 跟 bias
                這是一個 OrderedDict
        '''
        for block in [1, 2, 3, 4]:
            x = ConvBlockFunction(
                x,
                params[f'conv{block}.0.weight'],
                params[f'conv{block}.0.bias'],
                params.get(f'conv{block}.1.weight'),
                params.get(f'conv{block}.1.bias'))
        x = x.view(x.shape[0], -1)
        x = F.linear(x,
                     params['logits.weight'],
                     params['logits.bias'])
        return x

In [ ]:
def create_label(n_way, k_shot):
    return (torch.arange(n_way)
                 .repeat_interleave(k_shot)
                 .long())

# 我們試著產生 5 way 2 shot 的 label 看看
create_label(5, 2)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

In [ ]:
def MAML(model, 
         optimizer, x, 
         n_way, k_shot, 
         q_query, 
         loss_fn, 
         inner_train_step=1, inner_lr=0.4, train=True
   ): 
    """
    Args:
    x is the input omniglot images for a meta_step, 
        shape = [batch_size, 
                 n_way * (k_shot + q_query), 
                 1, 28, 28]
    n_way: 每個分類的 task 要有幾個 class
    k_shot: 每個類別在 training 的時候會有多少張照片
    q_query: 在 testing 時，每個類別會用多少張照片 update
    """

    criterion = loss_fn
    task_loss = []  # 這裡面之後會放入每個 task 的 loss
    task_acc = []   # 這裡面之後會放入每個 task 的 acc

    for meta_batch in x:
        # support_set 是我們拿來 update inner loop 
        #    參數的 data
        support_set = meta_batch[: n_way * k_shot]  
        # query_set 是我們拿來 update outer loop 
        #    參數的 data
        query_set = meta_batch[n_way * k_shot :]    
        
        # 在 inner loop update 參數時，我們不能動到實際
        #    參數，因此用 fast_weights 來儲存新的參數 θ'
        fast_weights = OrderedDict(
                             model.named_parameters())
        
        for inner_step in range(inner_train_step): 
            train_label = create_label(
                                 n_way, k_shot).cuda()
            logits = model.functional_forward(
                            support_set, fast_weights)
            loss = criterion(logits, train_label)

            # 這裡是要計算出 loss 對 θ 的微分 (∇loss)
            grads = torch.autograd.grad(
                loss, fast_weights.values(), 
                create_graph=True) 
            # 這裡是用剛剛算出的 ∇loss 
            #        來 update θ 變成 θ'
            fast_weights = OrderedDict(
                (name, param - inner_lr * grad)
                for ((name, param), grad) in zip(
                         fast_weights.items(), grads))
   
        val_label = create_label(
                                n_way, q_query).cuda()

        #$$ 一階微分，meta test
        #<<<<<<<<<< -------- (a) --------- >>>>>>>>>>#
        # 這裡用 query_set 和 θ' 算 logit              #
        logits = model.functional_forward(           #
                            query_set, fast_weights) #
        #............................................#
        # 這裡用 query_set 和 θ' 算 loss
        loss = criterion(logits, val_label)
        # 把這個 task 的 loss 丟進 task_loss 裡面
        task_loss.append(loss)
        # 算 accuracy
        acc = np.asarray([(
               torch.argmax(logits, -1).cpu().numpy()
            == val_label.cpu().numpy())]).mean() 
        task_acc.append(acc)

    model.train()
    optimizer.zero_grad()
    # 我們要用一整個 batch 的 loss 來 update θ (不是 θ')
    meta_batch_loss = torch.stack(task_loss).mean()
    if train:
        meta_batch_loss.backward()
        optimizer.step()
    task_acc = np.mean(task_acc)
    return meta_batch_loss, task_acc

In [21]:
def FOMAML(model, 
         optimizer, x, 
         n_way, k_shot, 
         q_query, 
         loss_fn, 
         inner_train_step=1, inner_lr=0.4, train=True
   ): 
    """
    Args:
    x is the input omniglot images for a meta_step, 
        shape = [batch_size, 
                 n_way * (k_shot + q_query), 
                 1, 28, 28]
    n_way: 每個分類的 task 要有幾個 class
    k_shot: 每個類別在 training 的時候會有多少張照片
    q_query: 在 testing 時，每個類別會用多少張照片 update
    """

    criterion = loss_fn
    task_loss = []  # 這裡面之後會放入每個 task 的 loss
    task_acc = []   # 這裡面之後會放入每個 task 的 acc

    for meta_batch in x:
        # support_set 是我們拿來 update inner loop 
        #    參數的 data
        support_set = meta_batch[: n_way * k_shot]  
        # query_set 是我們拿來 update outer loop 
        #    參數的 data
        query_set = meta_batch[n_way * k_shot :]    
        
        # 在 inner loop update 參數時，我們不能動到實際
        #    參數，因此用 fast_weights 來儲存新的參數 θ'
        fast_weights = OrderedDict(
                             model.named_parameters())
        
        for inner_step in range(inner_train_step): 
            train_label = create_label(
                                 n_way, k_shot).cuda()
            logits = model.functional_forward(
                            support_set, fast_weights)
            loss = criterion(logits, train_label)

            # 這裡是要計算出 loss 對 θ 的微分 (∇loss)
            grads = torch.autograd.grad(
                loss, fast_weights.values(), 
                create_graph=False) 
            # 這裡是用剛剛算出的 ∇loss 
            #        來 update θ 變成 θ'
            fast_weights = OrderedDict(
                (name, param - inner_lr * grad)
                for ((name, param), grad) in zip(
                         fast_weights.items(), grads))
   
        val_label = create_label(
                                n_way, q_query).cuda()

        #$$ 一階微分，meta test
        #<<<<<<<<<< -------- (a) --------- >>>>>>>>>>#
        # 這裡用 query_set 和 θ' 算 logit              #
        logits = model.functional_forward(           #
                            query_set, fast_weights) #
        #............................................#
        # 這裡用 query_set 和 θ' 算 loss
        loss = criterion(logits, val_label)
        # 把這個 task 的 loss 丟進 task_loss 裡面
        task_loss.append(loss)
        # 算 accuracy
        acc = np.asarray([(
               torch.argmax(logits, -1).cpu().numpy()
            == val_label.cpu().numpy())]).mean() 
        task_acc.append(acc)

    model.train()
    optimizer.zero_grad()
    # 我們要用一整個 batch 的 loss 來 update θ (不是 θ')
    meta_batch_loss = torch.stack(task_loss).mean()
    if train:
        meta_batch_loss.backward()
        optimizer.step()
    task_acc = np.mean(task_acc)
    return meta_batch_loss, task_acc

In [22]:
class Omniglot(Dataset):
    def __init__(self, data_dir, k_way, q_query):
        self.file_list = [f for f in glob.glob(
            data_dir + "**/character*", 
            recursive=True)]
        self.transform = transforms.Compose(
                            [transforms.ToTensor()])
        self.n = k_way + q_query
    def __getitem__(self, idx):
        sample = np.arange(20)
        # 這裡是為了等一下要 random sample 出我們要的    \
        #     character
        np.random.shuffle(sample) 
        img_path = self.file_list[idx]
        img_list = [f for f in glob.glob(
            img_path + "**/*.png", recursive=True)]
        img_list.sort()
        imgs = [self.transform(
            Image.open(img_file)) 
            for img_file in img_list]
        # 每個 character，取出 k_way + q_query 個
        imgs = torch.stack(imgs)[sample[:self.n]] 
        return imgs
    def __len__(self):
        return len(self.file_list)

In [23]:
""" >>> Start Training """;

In [24]:
n_way = 5
k_shot = 1
q_query = 1
inner_train_step = 1
inner_lr = 0.4
meta_lr = 0.001
meta_batch_size = 32
max_epoch = 80
eval_batches = test_batches = 20
train_data_path = './Omniglot/images_background/'
test_data_path = './Omniglot/images_evaluation/'    

In [25]:
NUM_W = 2
# dataset=Omniglot(train_data_path, k_shot, q_query)
train_set, val_set = torch.utils.data.random_split(
    Omniglot(train_data_path, k_shot, q_query),
    [3200, 656])
train_loader = DataLoader(train_set,
                          # 這裡的 batch_size 並不是  \
                          #     meta batch size, 而  \
                          #     是一個 task 裡面會有多 \
                          #     少不同的 characters， \
                          #     也就是 few-shot      \
                          #     classification 的    \
                          #     n_way
                          batch_size=n_way,
                          num_workers=NUM_W,
                          shuffle=True,
                          drop_last=True)
val_loader = DataLoader(val_set,
                        batch_size=n_way,
                        num_workers=NUM_W,
                        shuffle=True,
                        drop_last=True)
test_loader = DataLoader(Omniglot(
                             test_data_path,
                             k_shot, q_query),
                         batch_size=n_way,
                         num_workers=NUM_W,
                         shuffle=True,
                         drop_last=True)
train_iter = iter(train_loader)
val_iter = iter(val_loader)
test_iter = iter(test_loader)

In [26]:
meta_model = Classifier(1, n_way).cuda()
optimizer = torch.optim.Adam(meta_model.parameters(), 
                             lr=meta_lr)
loss_fn = nn.CrossEntropyLoss().cuda()

In [27]:
def get_meta_batch(meta_batch_size,
                   k_shot, q_query, 
                   data_loader, iterator):
    data = []
    for _ in range(meta_batch_size):
        try:
            # 一筆 task_data 就是一個 task 裡面的 data，\
            #     大小是                              \
            #     [n_way, k_shot+q_query, 1, 28, 28]
            task_data = iterator.next()  
        except StopIteration:
            iterator = iter(data_loader)
            task_data = iterator.next()
        train_data = (task_data[:, :k_shot]
                      .reshape(-1, 1, 28, 28))
        val_data = (task_data[:, k_shot:]
                    .reshape(-1, 1, 28, 28))
        task_data = torch.cat(
            (train_data, val_data), 0)
        data.append(task_data)
    return torch.stack(data).cuda(), iterator

<a name="mainprog" id="mainprog"></a>

In [28]:
from tqdm.auto import tqdm

coriginalMAML = MAML
coriginalMAML = FOMAML
# coriginalMAML = originalMAML
for epoch in range(max_epoch):
    print("Epoch %d" % (epoch + 1))
    train_meta_loss = []
    train_acc = []
    # 這裡的 step 是一次 meta-gradinet update step
    for step in tqdm(range(
            len(train_loader) // meta_batch_size)): 
        x, train_iter = get_meta_batch(
            meta_batch_size, k_shot, q_query, 
            train_loader, train_iter)
        meta_loss, acc = coriginalMAML(
        # meta_loss, acc = MAML(
            meta_model, optimizer, x, 
            n_way, k_shot, q_query, loss_fn)
        train_meta_loss.append(meta_loss.item())
        train_acc.append(acc)
    print("  Loss    : ", "%.3f" % (np.mean(train_meta_loss)), end='\t')
    print("  Accuracy: ", "%.3f %%" % (np.mean(train_acc) * 100))

    # 每個 epoch 結束後，看看 validation accuracy 如何  
    # 助教並沒有做 early stopping，                  \
    #     同學如果覺得有需要是可以做的 
    val_acc = []
    for eval_step in tqdm(range(
            len(val_loader) // (eval_batches))):
        x, val_iter = get_meta_batch(
            eval_batches, k_shot, q_query, 
            val_loader, val_iter)
        # testing時，我們更新三次 inner-step
        _, acc = coriginalMAML(meta_model, optimizer, x, 
        # _, acc = MAML(meta_model, optimizer, x, 
                      n_way, k_shot, q_query, 
                      loss_fn, 
                      inner_train_step=3, 
                      train=False) 
        val_acc.append(acc)
    print("  Validation accuracy: ", "%.3f %%" % (np.mean(val_acc) * 100))

Epoch 1



  Loss    :  1.831	  Accuracy:  34.031 %



  Validation accuracy:  51.167 %
Epoch 2



  Loss    :  1.503	  Accuracy:  39.531 %



  Validation accuracy:  55.667 %
Epoch 3


...


  Validation accuracy:  88.333 %
Epoch 79



  Loss    :  0.559	  Accuracy:  81.562 %



  Validation accuracy:  86.167 %
Epoch 80



  Loss    :  0.554	  Accuracy:  81.500 %



  Validation accuracy:  88.667 %


In [29]:
test_acc = []
for test_step in tqdm(range(
        len(test_loader) // (test_batches))):
    x, test_iter = get_meta_batch(
        test_batches, k_shot, q_query, 
        test_loader, test_iter)
    # testing 時，我們更新三次 inner-step
    _, acc = MAML(meta_model, optimizer, x, 
                  n_way, k_shot, q_query, loss_fn, 
                  inner_train_step=3, train=False)
    test_acc.append(acc)
print("  Testing accuracy: ", np.mean(test_acc))


  Testing accuracy:  0.8433333333333333


In [ ]:
raise

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!mkdir /content/drive/MyDrive/Ml15/
%cd /content/drive/MyDrive/Ml15/

In [ ]:
!nvidia-smi

In [ ]:
ls

In [ ]:
workspace_dir = '.'

In [ ]:
# gdown 是一個可以從 google drive 下載資料的工具
# gdown is a package that downloads files from       \
#     google drive
!gdown --id 1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U \
       --output "{workspace_dir}/Omniglot.tar.gz"

In [ ]:
# 使用 tar 解壓縮
# Use `tar' command to decompress
!tar -zxf "{workspace_dir}/Omniglot.tar.gz"          \
     -C "{workspace_dir}/"

In [ ]:
!rm -f Omniglot.tar.gz

In [ ]:
%cd ..

In [ ]:
rm -rf Omniglot/

In [ ]:
%cd /content/drive/MyDrive/Ml15

In [ ]:
from PIL import Image  # PIL 函式庫 / PIL library
from IPython.display import display
for i in range(10, 20):
    im = Image.open(
        "Omniglot/images_background/"
        "Japanese_(hiragana).0/"
        "character13/0500_" + str (i) + ".png")
    display(im)